# Построение бейзлайна.
---

В этом ноутбуке содержится код с построением бейзлайна. В качестве него нами была выбрана модель XGBoost со стандартными параметрами.

Грузим либы

In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import r2_score
from scipy.stats.mstats import winsorize
from tqdm.auto import tqdm
import os

np.random.seed(42)

## Данные

Загрузим датасет из соревнования. Обучение и валидацию будем проводить на train части. Часть test содержит в себе признаки без таргета, по ней будем строить предсказания для дальнейшей отправки в submission.

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

X_full = train.drop(columns=['id', 'FloodProbability'])
y_full = train['FloodProbability']
X_test = test.drop(columns=['id'])

print(f'Train shape: {train.shape}')
print(f'Test shape: {test.shape}')

Train shape: (1117957, 22)
Test shape: (745305, 21)


Реализуем функцию для винзоризации признаков.

In [3]:
def winsorize_features(df, limits=(0.01, 0.01)):
    df_win = df.copy()
    for col in tqdm(df.columns, desc='Winsorization', leave=False):
        df_win[col] = winsorize(df[col], limits=limits)
    return df_win

X_full_win = winsorize_features(X_full)
X_test_win = winsorize_features(X_test)

Winsorization:   0%|          | 0/20 [00:00<?, ?it/s]

Winsorization:   0%|          | 0/20 [00:00<?, ?it/s]

Масштабируем признаки с помощью робастного Scaler для устройчивости.

In [4]:
scaler = RobustScaler()
X_full_scaled = pd.DataFrame(
    scaler.fit_transform(X_full_win),
    columns=X_full_win.columns,
    index=X_full_win.index
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test_win),
    columns=X_test_win.columns,
    index=X_test_win.index
)

Генерируем новые признаки из оригинальных фичей датасета, добавляем их к масштабированным признакам.

In [5]:
def generate_baseline_features(df_orig, df_scaled):
    features = df_scaled.copy()

    features['sum'] = df_orig.sum(axis=1)
    features['mean'] = df_orig.mean(axis=1)
    features['std'] = df_orig.std(axis=1)
    features['max'] = df_orig.max(axis=1)
    features['min'] = df_orig.min(axis=1)
    features['median'] = df_orig.median(axis=1)
    features['range'] = features['max'] - features['min']
    features['q25'] = df_orig.quantile(0.25, axis=1)
    features['q75'] = df_orig.quantile(0.75, axis=1)
    features['iqr'] = features['q75'] - features['q25']
    features['cv'] = features['std'] / (features['mean'] + 1e-10)

    return features

X_full_feat = generate_baseline_features(X_full_win, X_full_scaled)
X_test_feat = generate_baseline_features(X_test_win, X_test_scaled)

Итого добавили 11 новых фичей, используя изученные техники.

## Модель

Инициализируем модель XGBoost в реализации из sklearn с базовыми параметрами.

In [6]:
xgb_model = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    n_jobs=-1,
    tree_method='hist'
)

Далее будем обучать модель, валидируя по стратегии RepeatedStratifiedKFold. Итоговую метрику получим усреднением.

In [7]:
# папка для выходных данных
os.makedirs('baseline_output', exist_ok=True)

# используем стратегию валидации RepeatedStratifiedKFold
n_repeats = 3
n_splits = 5
rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)

y_discrete = (y_full * 400).astype(np.int16)

oof_preds = np.zeros((len(X_full_feat), n_repeats))
test_preds = np.zeros((len(X_test_feat), n_repeats * n_splits))

total_folds = n_repeats * n_splits
fold_idx = 0

pbar = tqdm(total=total_folds, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}] {postfix}')

# процесс обучения
for repeat in range(n_repeats):
    for fold, (train_idx, val_idx) in enumerate(rskf.split(X_full_feat, y_discrete)):
        if fold >= n_splits * (repeat + 1):
            break
        if fold < n_splits * repeat:
            continue

        X_train, X_val = X_full_feat.iloc[train_idx], X_full_feat.iloc[val_idx]
        y_train, y_val = y_full.iloc[train_idx], y_full.iloc[val_idx]

        # обучаем
        xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

        oof_preds[val_idx, repeat] += xgb_model.predict(X_val)
        test_preds[:, fold_idx] = xgb_model.predict(X_test_feat)

        # считаем метрику
        val_r2 = r2_score(y_val, xgb_model.predict(X_val))
        pbar.set_postfix({'R²': f'{val_r2:.6f}'})
        pbar.update(1)

        fold_idx += 1

pbar.close()

# получаем итоговую метрику
oof_final = oof_preds.mean(axis=1)
final_r2 = r2_score(y_full, oof_final)

print(final_r2)

  0%|          | 0/15 [00:00<?] 

0.858277138606718


## Результаты

Итого мы получили ~0.85 на RepeatedStratifiedKFold валидации. Сохраняем результаты модели для использования в продвинутом решении.

In [8]:
np.save('baseline_output/baseline_predictions_train_oof.npy', oof_final)
np.save('baseline_output/baseline_predictions_test.npy', test_preds.mean(axis=1))

Строим файл для submission с предсказаниями на тесте в необходимой форме и сохраняем.

In [10]:
submission = pd.DataFrame({
    'id': test['id'],
    'FloodProbability': test_preds.mean(axis=1)
})

submission.to_csv('results/baseline.csv', index=False)